In [1]:
### Multiple

### Multistep Methods

- Euler's and Runge-Kutta methods are single-step approaches, in that they only use information at $x(t)$ to determine its value at the next time step

- Multistep methods take advantage of the fact that using information about previous time steps $x(t-\Delta t), x(t-2\Delta t)$, etc

- These methods can be explicit or implicit (dependent on $x(t + \Delta t)$ values); we'll just consider explicit Adams-Bashforth approach

### Multistep Motivation

- In determining $x(t + \Delta t)$, we can use a Taylor series expansion about $x(t)$

$$
x(t + \Delta t) = x(t) + \Delta t x'(t) + \frac{\Delta t^2}{2} x''(t) + O(\Delta t^3)
$$

$$
x(t + \Delta t) = x(t) + \Delta t f(x(t), t) + \frac{\Delta t^2}{2} \Biggl\{\frac{f(x(t)) - f(x(t - \Delta t))}{\Delta t} + O(\Delta t)\Biggl\}
$$

(note Euler's method is just the first two terms on the right-hand side)

$$
x(t + \Delta t) = x(t) + \Delta t \Biggl\{\frac{3}{2} f(x(t)) - \frac{1}{2} f(x(t - \Delta t))\Biggl\} + O(\Delta t^3)
$$

- What we derived is a second-order Adams-Bashforth method. Higher-order methods can be developed in a similar fashion, but approximating subsequent derivatives.

- A key Adams-Bashforth method is the approach only requires one function evaluation per time step, as opposed to Runge-Kutta methods which require multiple evaluations.


### Stiff Differential Equations

- Stiff differential equations are ones in which the desired solution has components the vary quite rapidly relative to the solution

- Stifness is associated with solution efficiency: in order to account for these fast dynamics we need to use very small time steps

$$
x_1^{\prime} = x_2\
$$

$$
x_2^{\prime} = -1000 x_1 - 1001 x_2
$$

$$
x^{\prime} = \begin{bmatrix} 0 & 1 \\ -1000 & -1001 \end{bmatrix} x
$$

$$
x_1(t) = A e^{-t} + B e^{-1001 t}
$$

(note that solution $x_1(t)$ is not validated, and maybe incorrect)

### Implicit Methods

- Implicit methods have the advantage of being numerically stable over the entire left half plane

- Only mehtods considered here are the Backward Euler and Trapezoidal methods

$$
x^{\prime} = f(x(t)) = A x(t)
$$

Initially we'll assume linear equations

Then using backward Euler method, we have

$$
x(t + \Delta t) = x(t) + \Delta t A x(t + \Delta t)
$$

$$
(I - \Delta t A) x(t + \Delta t) = x(t)
$$

$$
x(t + \Delta t) = (I - \Delta t A)^{-1} x(t)
$$

### Backward Euler Cart Example
use the same example as before

In [2]:
import numpy as np
import scipy.linalg as la
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(True)

def backward_euler_step_linear(A, x, dt):
    return np.linalg.inv(np.eye(len(A)) - dt * A).dot(x)

def backward_euler_linear(A, x0, t0, t_end, dt):
    x = x0
    t = t0
    sol = [x0]
    while t < t_end:
        x = backward_euler_step_linear(A, x, dt)
        sol.append(x)
        t = t + dt
    return np.array(sol)

A = np.array([[0, 1], [-1, 0]])
x0 = np.array([1, 0])
t0 = 0
t_end = 2
dt = 0.25

t_span1 = np.arange(t0, t_end + dt, dt)
sol1 = backward_euler_linear(A, x0, t0, t_end, dt)
dt = 0.05
t_span2 = np.arange(t0, t_end + dt, dt)
sol_exact = np.array([la.expm(A * t).dot(x0) for t in t_span2])
sol2 = backward_euler_linear(A, x0, t0, t_end, dt)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t_span2, y=sol_exact[:, 0], mode='lines', name='Exact x1'))
fig.add_trace(go.Scatter(x=t_span2, y=sol_exact[:, 1], mode='lines', name='Exact x2'))
fig.add_trace(go.Scatter(x=t_span1, y=sol1[:, 0], mode='lines', name='Backward Euler dt=0.25 x1'))
fig.add_trace(go.Scatter(x=t_span1, y=sol1[:, 1], mode='lines', name='Backward Euler dt=0.25 x2'))
fig.add_trace(go.Scatter(x=t_span2, y=sol2[:, 0], mode='lines', name='Backward Euler dt=0.05 x1'))
fig.add_trace(go.Scatter(x=t_span2, y=sol2[:, 1], mode='lines', name='Backward Euler dt=0.05 x2'))
iplot(fig)

### Trapezoidal Method

- For the trapezoidal with a linear system we have

$$
x(t + \Delta t) = x(t) + \frac{\Delta t}{2} \Biggl\{A x(t) + A x(t + \Delta t)\Biggl\}
$$

$$
(I - \frac{\Delta t}{2} A) x(t + \Delta t) = (I + \frac{\Delta t}{2} A) x(t)
$$

$$
x(t + \Delta t) = (I - \frac{\Delta t}{2} A)^{-1} (I + \frac{\Delta t}{2} A) x(t)
$$


In [3]:
import numpy as np
import scipy.linalg as la
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(True)

def trapezoidal_step_linear(A, x, dt):
    return np.linalg.inv(np.eye(len(A)) - dt / 2 * A).dot(np.eye(len(A)) + dt / 2 * A).dot(x)

def trapezoidal_linear(A, x0, t0, t_end, dt):
    x = x0
    t = t0
    sol = [x0]
    while t < t_end:
        x = trapezoidal_step_linear(A, x, dt)
        sol.append(x)
        t = t + dt
    return np.array(sol)

A = np.array([[0, 1], [-1, 0]])
x0 = np.array([1, 0])
t0 = 0
t_end = 2
dt = 0.25

t_span1 = np.arange(t0, t_end + dt, dt)
sol1 = trapezoidal_linear(A, x0, t0, t_end, dt)
dt = 0.05
t_span2 = np.arange(t0, t_end + dt, dt)
sol2 = trapezoidal_linear(A, x0, t0, t_end, dt)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t_span2, y=sol_exact[:, 0], mode='lines', name='Exact x1'))
fig.add_trace(go.Scatter(x=t_span2, y=sol_exact[:, 1], mode='lines', name='Exact x2'))
fig.add_trace(go.Scatter(x=t_span1, y=sol1[:, 0], mode='lines', name='Trapezoidal dt=0.25 x1'))
fig.add_trace(go.Scatter(x=t_span1, y=sol1[:, 1], mode='lines', name='Trapezoidal dt=0.25 x2'))
fig.add_trace(go.Scatter(x=t_span2, y=sol2[:, 0], mode='lines', name='Trapezoidal dt=0.05 x1'))
fig.add_trace(go.Scatter(x=t_span2, y=sol2[:, 1], mode='lines', name='Trapezoidal dt=0.05 x2'))
iplot(fig)

### Transmission Line Modeling

- In power flow and transient stability, transmission lines are typically modeled using a lumped parameter approach, where the changes in voltage and current are assumed to occur instantaneously

- In EMTP time-frame, this is no longer the case considering the speed of light

### Incremental Transmission Line Modeling

![Lecture3_1.png](./assets/Lect3_1.png)

$$
\Delta v = R'\Delta x i + L'\Delta x \frac{\partial i}{\partial t}
$$

$$
\Delta i = G'\Delta x(v + \Delta v) + C'\Delta x \frac{\partial}{\partial t}(v + \Delta v)
$$

After some derivations, we finally have the goal, the model of the transmission at the sending end and receiving end could be modeled as two separate Norton equivalent circuits, where the corresponding current source is time-varying

![Lecture3_2.png](./assets/Lect3_2.png)

- the model of transmission line is a model dependent on the time, suitable for numerical studies on this time frame

$$
I_k(t) = i_m\left(t - \frac{d}{v_p}\right) - \frac{1}{z_c}v_m\left(t - \frac{d}{v_p}\right)
$$

$$
I_m(t) = i_k\left(t - \frac{d}{v_p}\right) + \frac{1}{z_c}v_k\left(t - \frac{d}{v_p}\right)
$$

Thus, the sending end and receiving end can be represented in circuit form as

$$
i_k(t) = \frac{v_k(t)}{z_c} + I_k(t)
$$

$$
i_m(t) = - \frac{v_m(t)}{z_c} + I_m(t)
$$

### Numerical Intergration with Trapezoidal Method applied into Inductors and Capacitors

- For a general function the trapezoidal method is numerically stable as follows

$$
x^{\prime}(t) = f(x(t), t)
$$

$$
x(t + \Delta t) = x(t) + \frac{\Delta t}{2} \Biggl\{f(x(t)) + f(x(t + \Delta t))\Biggl\}
$$

- For an inductor, we have a linear equation as follows

$$
v = L \frac{di}{dt}  => \frac{di}{dt} = \frac{v}{L}, i(0) = i_0
$$

$$
i(t + \Delta t) = i(t) + \frac{\Delta t}{2L} \Biggl\{v(t) + v(t + \Delta t)\Biggl\}
$$

This can be represented as a Norton equivalent with current into the equivalent defined as positive (the last two terms are the current source)

$$
i_k(t + \Delta t) = \frac{v(t + \Delta t)}{2L / \Delta t} + i(t) + \frac{v(t)}{2L / \Delta t}
$$

- For an inductor in series with a resistor, we have

$$
v = R i + L \frac{di}{dt} => \frac{di}{dt} = -\frac{R}{L} i + \frac{1}{L} v, i(0) = i_0
$$

$$
i(t + \Delta t) = i(t) + \frac{\Delta t}{2L} \Biggl\{-R i(t) + v(t) - R i(t + \Delta t) + v(t + \Delta t)\Biggl\}
$$

Then

$$
i(t + \Delta t) = \frac{i(t) + \frac{\Delta t}{2L} \Biggl\{-R i(t) + v(t) + v(t + \Delta t)\Biggl\}}{1 + \frac{\Delta t}{2L} R}
$$



### RL Example

- Assume a series RL circuit with an open switch with $R = 200 \Omega$ and $L = 0.3 H$, connected to a voltage source with $v = 133,000 \sqrt{2} \cos(2\pi 60 t)$

- Assume the switch is closed at t = 0

- The exact solution is

$$
i(t) = -712.4 e^{-667 t} + 578.8 \sqrt{2} \cos(2\pi 60 t - 29.5^\circ)
$$

In [4]:
import numpy as np
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(True)

def v_source(V_mag, f):
    return lambda t: V_mag * np.cos(2 * np.pi * f * t)


def rl_circuit(R, L, v_source, t_end, dt, i0, t0):
    i = i0
    t = t0
    sol = [i0]
    while t < t_end:
        i = i + dt / (2 * L) * (-R * i + v_source(t) - R * i + v_source(t + dt)) / (1 + dt / (2 * L) * R)
        sol.append(i)
        t = t + dt
    return np.array(sol)

R = 200
L = 0.3
v_source = v_source(133000 * np.sqrt(2), 60)
t_end = 0.05
dt = 0.0001
i0 = 0
t0 = 0

sol = rl_circuit(R, L, v_source, t_end, dt, i0, t0)
sol_exact = np.array([-712.4 * np.exp(-667 * t) + 578.8 * np.sqrt(2) * np.cos(2 * np.pi * 60 * t - 29.5 * np.pi / 180) for t in np.arange(t0, t_end + dt, dt)])

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(t0, t_end + dt, dt), y=sol, mode='lines', name='Numerical'))
fig.add_trace(go.Scatter(x=np.arange(t0, t_end + dt, dt), y=sol_exact, mode='lines', name='Exact'))
iplot(fig)

### Lumped Capacitance Model

- The trapezoidal approach can also be applied into model lumped capacitors

$$
i(t) = C \frac{dv(t)}{dt}
$$

- Integrating over a time step gives

$$
v(t + \Delta t) = v(t) + \frac{\Delta t}{2C} \Biggl\{i(t) + i(t + \Delta t)\Biggl\}
$$

$$
i(t + \Delta t) = \frac{v(t + \Delta t)}{\Delta t / 2 C} - \frac{v(t)}{\Delta t / 2 C} - i(t)
$$

- It will become a odes with two states if we include the inductor and capacitor together

### Example 2.1: Line closing

![Lecture3_3.png](./assets/Lect3_3.png)

- Initial conditions: $i_1 = i_2 = v_1 = v_2 = 0$
- for $t \lt 0.0001$ sec
$$
z_c = \sqrt{\frac{L^{\prime}}{C^{\prime}}} = 274 \Omega
$$

$$
v_p = \frac{1}{\sqrt{L^{\prime}C^{\prime}}} = 182,574 \text{ mi/sec}
$$

$$
\frac{d}{v_p} = 0.00055 \text{ sec}
$$

$$
\frac{2 L}{\Delta t} = \frac{2 * 0.25}{0.0001} = 5000 \Omega
$$

- Consider the sending end and receiving end of the line connected to a RL load, we have two circuits.

![Lecture3_4.png](./assets/Lect3_4.png)

- For the sending end circuit, we have

$$
v_1(t + \Delta t) = v_source(t + \Delta t) \quad \text{(extra voltage constraint equation for the sending end circuit)}
$$

$$
i_1(t + \Delta t) = \frac{v_1(t + \Delta t - \frac{d}{v_p})}{z_c} + i_2\left(t + \Delta t - \frac{d}{v_p}\right) - \frac{1}{z_c}v_2\left(t + \Delta t - \frac{d}{v_p}\right)
$$

- For the receiving end circuit, we have

$$
v_2(t + \Delta t) = R i_2(t + \Delta t) + v_3(t + \Delta t) \quad \text{(extra voltage constraint equation for the receiving end circuit due to the RL load)}
$$

$$
i_2(t + \Delta t) = - \frac{v_2(t + \Delta t)}{z_c} + i_1\left(t + \Delta t - \frac{d}{v_p}\right) + \frac{1}{z_c}v_1\left(t + \Delta t - \frac{d}{v_p}\right)
$$

- For the RL load, we have

$$
i_2(t + \Delta t) = i_2(t) + \frac{v_3(t)}{2L / \Delta t} + \frac{v_3(t + \Delta t)}{2L / \Delta t}
$$

Thus, we have a system of 5 algebraic equations with 5 unknowns, $v_1(t + \Delta t), i_1(t + \Delta t), v_2(t + \Delta t), i_2(t + \Delta t), v_3(t + \Delta t)$

In [5]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import fsolve
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(True)

class Variable(dict):
    def __init__(self, y0, t0):
        super().__init__()
        self.t0 = t0
        self[t0] = y0

    def __getitem__(self, key):
        if key < self.t0:
            return 0
        # use linear interpolation if the value is not in the solution dictionary
        if key not in self:
            return np.interp(key, np.array(list(self.keys())), np.array(list(self.values())))
        return super().__getitem__(key)

def v_source(V_mag, f):
    return lambda t: V_mag * np.cos(2 * np.pi * f * t)

def line_closing(R, L, L_prime, C_prime, d, v_source, t_end, dt, y0, t0):
    z_c = np.sqrt(L_prime / C_prime)
    v_p = 1 / np.sqrt(L_prime * C_prime)
    d_over_vp = d / v_p
    two_L_over_dt = 2 * L / dt

    v1, i1, v2, i2, v3 = Variable(y0[0], t0), Variable(y0[1], t0), Variable(y0[2], t0), Variable(y0[3], t0), Variable(y0[4], t0)
    t = t0

    # circuit 1
    # the following two equations with two unknowns, v1[t + dt], i1[t + dt]
    # v1[t + dt] = v_source(t + dt)
    # i1[t + dt] = v1[t + dt] / z_c + i2.get(t - d_over_vp, 0) - 1 / z_c * v2.get(t - d_over_vp, 0)
    # circuit 2
    # the following three equations with three unknowns, i2[t + dt], v2[t + dt], i3[t + dt]
    # i2[t + dt] = i2[t] + v3[t] / two_L_over_dt + v3[t + dt] / two_L_over_dt
    # v2[t + dt] = (R * i2[t + dt]) + v3[t + dt]
    # i2[t + dt] = -v2[t + dt] / z_c + i1.get(t - d_over_vp, 0) + 1 / z_c * v1.get(t - d_over_vp, 0)

    def circuit_eqs(y, t, dt, R, two_L_over_dt, z_c, d_over_vp, v_source, v1, i1, v2, i2, v3):
        next_v1, next_i1, next_v2, next_i2, next_v3 = y
        return [-next_v1 + v_source(t + dt),
                -next_i1 + next_v1 / z_c + i2[t - d_over_vp] - 1 / z_c * v2[t - d_over_vp],
                -next_i2 + i2[t] + v3[t] / two_L_over_dt + next_v3 / two_L_over_dt,
                -next_v2 + R * next_i2 + next_v3,
                -next_i2 - next_v2 / z_c + i1[t - d_over_vp] + 1 / z_c * v1[t - d_over_vp]]
    
    while t < t_end - 1e-12:
        sol = fsolve(circuit_eqs, [v1[t], i1[t], v2[t], i2[t], v3[t]], args=(t, dt, R, two_L_over_dt, z_c, d_over_vp, v_source, v1, i1, v2, i2, v3))
        v1[t + dt], i1[t + dt], v2[t + dt], i2[t + dt], v3[t + dt] = sol
        t = t + dt
    return v1, i1, v2, i2, v3

R = 400
L = 0.25
L_prime = 1.5e-3
C_prime = 0.02e-6
d = 100

v_source = v_source(230000 * np.sqrt(2/3), 60)

t0 = 0
t_end = 0.05
dt = 0.0001

v1, i1, v2, i2, v3 = line_closing(R, L, L_prime, C_prime, d, v_source, t_end, dt, [0, 0, 0, 0, 0], t0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(t0, t_end + dt, dt), y=list(v1.values()), mode='lines', name='v1'))
# fig.add_trace(go.Scatter(x=np.arange(t0, t_end + dt, dt), y=list(i1.values()), mode='lines', name='i1'))
fig.add_trace(go.Scatter(x=np.arange(t0, t_end + dt, dt), y=list(v2.values()), mode='lines', name='v2'))
# fig.add_trace(go.Scatter(x=np.arange(t0, t_end + dt, dt), y=list(i2.values()), mode='lines', name='i2'))
# fig.add_trace(go.Scatter(x=np.arange(t0, t_end + dt, dt), y=list(v3.values()), mode='lines', name='v3'))
iplot(fig)